In [17]:
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/home/karvsmech/Projects/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.options.display.max_columns = 100
pd.options.display.max_rows =  200
pd.options.display.max_info_rows = 1690785
pd.options.display.max_info_columns = 200
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.date_dayfirst = True
pd.options.mode.chained_assignment = None    

/home/karvsmech/Projects/kaggle/input/house_prices_prediction/data_description.txt
/home/karvsmech/Projects/kaggle/input/house_prices_prediction/test.csv
/home/karvsmech/Projects/kaggle/input/house_prices_prediction/.~lock.train.csv#
/home/karvsmech/Projects/kaggle/input/house_prices_prediction/train.csv
/home/karvsmech/Projects/kaggle/input/house_prices_prediction/sample_submission.csv


In [2]:
df = pd.read_csv('/home/karvsmech/Projects/kaggle/input/house_prices_prediction/train.csv')
df.head(5)

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL        65.00     8450   Pave   NaN      Reg   
1   2          20       RL        80.00     9600   Pave   NaN      Reg   
2   3          60       RL        68.00    11250   Pave   NaN      IR1   
3   4          70       RL        60.00     9550   Pave   NaN      IR1   
4   5          60       RL        84.00    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5       2003   
1       Norm     1Fam     1Story            6            8       1976   
2       Norm     1Fam     2Story            7            5       2001   
3       Norm     1Fam     2Story            7            5       1915   
4       Norm     1Fam     2Story            8            5       2000   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1          1976     Gable  CompShg     MetalSd     MetalSd        NaN   
2          2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3          1970     Gable  CompShg     Wd Sdng     Wd Shng        NaN   
4          2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0      196.00        Gd        TA      PConc       Gd       TA           No   
1        0.00        TA        TA     CBlock       Gd       TA           Gd   
2      162.00        Gd        TA      PConc       Gd       TA           Mn   
3        0.00        TA        TA     BrkTil       TA       Gd           No   
4      350.00        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0         NaN   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1          Gd            

In [3]:
dtypes = df.dtypes.to_frame().reset_index()
dtypes.columns = ['col', 'dtype']
dtypes.groupby('dtype').size()

dtype
int64      35
float64     3
object     43
dtype: int64

In [5]:
def na(df, percent=True, verbose=True):
    na_series = df.isna().sum()
    na_series = na_series.where(na_series > 0).dropna().sort_values(ascending=False)
    if percent:
        if verbose:
            print("% of NaNs in each column")
        return (na_series/df.shape[0]) * 100
    else:
        if verbose:
            print("# of NaNs in each column:")
        return na_series
    
na(df, False)

# of NaNs in each column:


PoolQC         1,453.00
MiscFeature    1,406.00
Alley          1,369.00
Fence          1,179.00
MasVnrType       872.00
FireplaceQu      690.00
LotFrontage      259.00
GarageType        81.00
GarageYrBlt       81.00
GarageFinish      81.00
GarageQual        81.00
GarageCond        81.00
BsmtFinType2      38.00
BsmtExposure      38.00
BsmtFinType1      37.00
BsmtCond          37.00
BsmtQual          37.00
MasVnrArea         8.00
Electrical         1.00
dtype: float64

In [6]:
sns.heatmap(df.select_dtypes(exclude='O').corr(), center=0)

<Axes: >

In [7]:
def get_corr_list(df, threshold = 0.6):
    df_corr = df[num_vars].corr().unstack().to_frame().reset_index()
    df_corr.columns = ['var1', 'var2', 'corr']

    # removing rows where var1 is equal to var2
    df_corr = df_corr.loc[df_corr['var1'] != df_corr['var2'], :]
    df_corr['abs_corr'] = df_corr['corr'].abs()
    df_corr = df_corr.loc[df_corr['abs_corr'] > threshold, :]
    df_corr.sort_values(by=['abs_corr'], ascending=False, inplace=True)

    #remove mirrored items
    df_corr = df_corr.iloc[::2]
    
    return df_corr.reset_index()
    
    
num_vars = df.select_dtypes(include='number').columns.tolist()
df_corr = get_corr_list(df[num_vars])
df_corr

index          var1          var2  corr  abs_corr
0    1015    GarageCars    GarageArea  0.88      0.88
1     956   GarageYrBlt     YearBuilt  0.83      0.83
2     631     GrLivArea  TotRmsAbvGrd  0.83      0.83
3     469   TotalBsmtSF      1stFlrSF  0.82      0.82
4     189   OverallQual     SalePrice  0.79      0.79
5     645     GrLivArea     SalePrice  0.71      0.71
6     548      2ndFlrSF     GrLivArea  0.69      0.69
7     821  BedroomAbvGr  TotRmsAbvGrd  0.68      0.68
8     655  BsmtFullBath    BsmtFinSF1  0.65      0.65
9     957   GarageYrBlt  YearRemodAdd  0.64      0.64
10   1025    GarageCars     SalePrice  0.64      0.64
11    627     GrLivArea      FullBath  0.63      0.63
12   1433     SalePrice    GarageArea  0.62      0.62
13    888  TotRmsAbvGrd      2ndFlrSF  0.62      0.62
14   1418     SalePrice   TotalBsmtSF  0.61      0.61
15    774      HalfBath      2ndFlrSF  0.61      0.61
16   1419     SalePrice      1stFlrSF  0.61      0.61
17    992    GarageCars   OverallQual  0.60      0.60

In [8]:
__neighborhood = df.groupby('Neighborhood').agg({'Neighborhood':'size', 'SalePrice':'mean'})
__neighborhood.sort_values('SalePrice')

Neighborhood  SalePrice
Neighborhood                         
MeadowV                 17  98,576.47
IDOTRR                  37 100,123.78
BrDale                  16 104,493.75
BrkSide                 58 124,834.05
Edwards                100 128,219.70
OldTown                113 128,225.30
Sawyer                  74 136,793.14
Blueste                  2 137,500.00
SWISU                   25 142,591.36
NPkVill                  9 142,694.44
NAmes                  225 145,847.08
Mitchel                 49 156,270.12
SawyerW                 59 186,555.80
NWAmes                  73 189,050.07
Gilbert                 79 192,854.51
Blmngtn                 17 194,870.88
CollgCr                150 197,965.77
Crawfor                 51 210,624.73
ClearCr                 28 212,565.43
Somerst                 86 225,379.84
Veenker                 11 238,772.73
Timber                  38 242,247.45
StoneBr                 25 310,499.00
NridgHt                 77 316,270.62
NoRidge                 41 335,295.32

In [12]:
__condition1 = df.groupby('Condition1').agg({'Condition1':'size', 'SalePrice':'mean'})
__condition1.sort_values('SalePrice')

Condition1  SalePrice
Condition1                       
Artery              48 135,091.67
RRAe                11 138,400.00
Feedr               81 142,475.48
RRAn                26 184,396.62
Norm              1260 184,495.49
RRNe                 2 190,750.00
RRNn                 5 212,400.00
PosN                19 215,184.21
PosA                 8 225,875.00

In [11]:
__condition2 = df.groupby('Condition2').agg({'Condition2':'size', 'SalePrice':'mean'})
__condition2.sort_values('SalePrice')

Condition2  SalePrice
Condition2                       
RRNn                 2  96,750.00
Artery               2 106,500.00
Feedr                6 121,166.67
RRAn                 1 136,905.00
Norm              1445 181,169.41
RRAe                 1 190,000.00
PosN                 2 284,875.00
PosA                 1 325,000.00

In [16]:
cat_cols = df.select_dtypes('O').columns.to_list() + ['MSSubClass']
cat_cols = [col for col in cat_cols if col not in ['Neighborhood', 'Condition1', 'Condition2', 'FireplaceQu']]
print(cat_cols)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'MSSubClass']


In [19]:
# OneHotEncoder
ohencoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
ohencoder.set_output(transform='pandas')

df_cats_encoded = ohencoder.fit_transform(df[cat_cols])
df_cats_encoded

MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Pave  \
0            0.00         0.00         1.00         0.00         1.00   
1            0.00         0.00         1.00         0.00         1.00   
2            0.00         0.00         1.00         0.00         1.00   
3            0.00         0.00         1.00         0.00         1.00   
4            0.00         0.00         1.00         0.00         1.00   
...           ...          ...          ...          ...          ...   
1455         0.00         0.00         1.00         0.00         1.00   
1456         0.00         0.00         1.00         0.00         1.00   
1457         0.00         0.00         1.00         0.00         1.00   
1458         0.00         0.00         1.00         0.00         1.00   
1459         0.00         0.00         1.00         0.00         1.00   

      Alley_Pave  Alley_nan  LotShape_IR2  LotShape_IR3  LotShape_Reg  \
0           0.00       1.00          0.00          0.00          1.00   
1           0.00       1.00          0.00          0.00          1.00   
2           0.00       1.00          0.00          0.00          0.00   
3           0.00       1.00          0.00          0.00          0.00   
4           0.00       1.00          0.00          0.00          0.00   
...          ...        ...           ...           ...           ...   
1455        0.00       1.00          0.00          0.00          1.00   
1456        0.00       1.00          0.00          0.00          1.00   
1457        0.00       1.00          0.00          0.00          1.00   
1458        0.00       1.00          0.00          0.00          1.00   
1459        0.00       1.00          0.00          0.00          1.00   

      LandContour_HLS  LandContour_Low  LandContour_Lvl  Utilities_NoSeWa  \
0                0.00             0.00             1.00              0.00   
1                0.00             0.00             1.00              0.00   
2                0.00             0.00             1.00              0.00   
3                0.00             0.00             1.00              0.00   
4                0.00             0.00             1.00              0.00   
...               ...              ...              ...               ...   
1455             0.00             0.00             1.00              0.00   
1456             0.00             0.00             1.00              0.00   
1457             0.00             0.00             1.00              0.00   
1458             0.00             0.00             1.00              0.00   
1459             0.00             0.00             1.00              0.00   

      LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside  \
0                  0.00           0.00           0.00              1.00   
1                  0.00           1.00           0.00              0.00   
2                  0.00           0.00           0.00              1.00   
3                  0.00           0.00           0.00              0.00   
4                  0.00           1.00           0.00              0.00   
...                 ...            ...            ...               ...   
1455               0.00           0.00           0.00              1.00   
1456               0.00           0.00           0.00              1.00   
1457               0.00           0.00           0.00              1.00   
1458               0.00           0.00           0.00              1.00   
1459               0.00           0.00           0.00              1.00   

      LandSlope_Mod  LandSlope_Sev  BldgType_2fmCon  BldgType_Duplex  \
0              0.00           0.00             0.00             0.00   
1              0.00           0.00             0.00             0.00   
2              0.00           0.00             0.00             0.00   
3              0.00           0.00             0.00             0.00   
4              0.00           0.00             0.00             0.